<a href="https://colab.research.google.com/github/amitkbiswas0/ocr-cnn/blob/master/ocr_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os, glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils import np_utils, plot_model
from keras.models import Model
from keras.layers import Input, Dropout, Dense, Flatten, BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
np.random.seed(0)
# data plotting
get_ipython().run_line_magic('matplotlib', 'inline')
sns.set(style='white', context='notebook', palette='deep')

In [13]:
path = '/content/drive/My Drive/Colab Notebooks/data/'
os.chdir(path)
extension_name = "csv"
all_files = [i for i in glob.glob("*.{}".format(extension_name))]
# print(all_files)
dataframe = pd.concat([pd.read_csv(f) for f in all_files])
print(dataframe.shape)
# df = pd.read_csv(path)
# print(df.shape)

(186735, 785)


In [14]:
labels = dataframe[["label"]].copy()
dataframe.pop("label")
features = dataframe
print(features.shape, labels.shape)

(186735, 784) (186735, 1)


In [15]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(149388, 784) (37347, 784) (149388, 1) (37347, 1)


In [0]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [17]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(y_train.shape, y_test.shape)
print(x_train.shape, x_test.shape)

(149388, 122) (37347, 122)
(149388, 784) (37347, 784)


In [18]:
x_train = np.reshape(x_train, (-1,28,28,1))
x_test = np.reshape(x_test, (-1,28,28,1))
print(x_train.shape)
print(x_test.shape)

(149388, 28, 28, 1)
(37347, 28, 28, 1)


In [19]:
# input layer
visible = Input(shape=(28,28,1))

# first 2 conv2d
layer_1 = Conv2D(32, kernel_size=3, padding="same", activation='relu')(visible)
layer_2 = Conv2D(32, 5, padding="same", activation="relu")(layer_1)
layer_3 = MaxPooling2D(pool_size=(2,2))(layer_2)
layer_4 = Dropout(rate=0.25)(layer_3)

# parallel part
# top layers
layer_5 = Conv2D(64, kernel_size=5, padding="same", activation="relu")(layer_4)
layer_6 = BatchNormalization()(layer_5)
layer_7 = Conv2D(64, kernel_size=5, padding="same", activation="relu")(layer_6)
layer_8 = BatchNormalization()(layer_7)
layer_9 = MaxPooling2D(pool_size=(2,2))(layer_8)
layer_10 = Dropout(rate=0.2)(layer_9)
# bottom layers
layer_11 = Conv2D(64, kernel_size=3, padding="same", activation="relu")(layer_4)
layer_12 = Conv2D(64, kernel_size=3, padding="same", activation="relu")(layer_11)
layer_13 = MaxPooling2D(pool_size=(2,2))(layer_12)
layer_14 = Dropout(rate=0.25)(layer_13)

# last part
layer_15 = concatenate([layer_10, layer_14])
layer_16 = Conv2D(64, kernel_size=3, padding="same", activation="relu")(layer_15)
layer_17 = MaxPooling2D(pool_size=(2,2))(layer_16)
layer_18 = Dropout(rate=0.25)(layer_17)

# rest
flatten = Flatten()(layer_18)
hidden = Dense(2048, activation="relu")(flatten)
layer_19 = Dropout(rate=0.25)(hidden)
output = Dense(122, activation="softmax")(layer_19)
model = Model(inputs=visible, outputs=output)

print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 28, 28, 32)   25632       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 14, 14, 32)   0           conv2d_2[0][0]                   
____________________________________________________________________________________________

In [0]:
optimizer = Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
# model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=50)